In [8]:
import pandas as pd
import requests
from tqdm import tqdm
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime

In [9]:
# Load wallets.csv
wallets_df = pd.read_csv("wallets.csv") 
wallets = wallets_df['wallet_id'].tolist()

print(f"Loaded {len(wallets)} wallet addresses.")


Loaded 103 wallet addresses.


In [4]:
# Define constants
COVALENT_API_KEY = "cqt_rQq3yVrGM37pKFY9fYfFRCcCj99v"
CHAIN_ID = "1"  # Ethereum Mainnet
base_url = f"https://api.covalenthq.com/v1/{CHAIN_ID}/address"

# Fetch Compound interactions (example: cETH token transfers)
def fetch_compound_tx(wallet):
    url = f"{base_url}/{wallet}/transactions_v2/?key={COVALENT_API_KEY}&page-size=10000"
    try:
        response = requests.get(url)
        data = response.json()
        return data['data']['items']
    except Exception as e:
        print(f"Error for wallet {wallet}: {e}")
        return []


In [10]:
def extract_features(tx_list):
    if not tx_list:
        return {
            "num_tx": 0,
            "total_eth_sent": 0,
            "total_eth_received": 0,
            "compound_tx_count": 0,
            "avg_gas_fee": 0,
            "tx_duration_days": 0
        }

    sent, received, compound_count, gas_fees = 0, 0, 0, []

    tx_times = []
    for tx in tx_list:
        value = int(tx['value']) / 1e18
        if tx['from_address_label'] is not None and "compound" in tx['from_address_label'].lower():
            compound_count += 1
        if tx['from_address'] == tx['to_address']:
            continue
        elif tx['from_address'] == wallet:
            sent += value
        elif tx['to_address'] == wallet:
            received += value
        gas_fees.append(float(tx['gas_spent']) * float(tx['gas_price']) / 1e18)
        tx_times.append(datetime.strptime(tx['block_signed_at'], "%Y-%m-%dT%H:%M:%SZ"))

    if tx_times:
        duration = (max(tx_times) - min(tx_times)).days
    else:
        duration = 0

    return {
        "num_tx": len(tx_list),
        "total_eth_sent": sent,
        "total_eth_received": received,
        "compound_tx_count": compound_count,
        "avg_gas_fee": np.mean(gas_fees) if gas_fees else 0,
        "tx_duration_days": duration
    }


In [6]:
features = []

for wallet in tqdm(wallets):
    txs = fetch_compound_tx(wallet)
    wallet_features = extract_features(txs)
    wallet_features['wallet_id'] = wallet
    features.append(wallet_features)

df = pd.DataFrame(features)
df.to_csv("compound_wallet_features.csv", index=False)
print("Feature file saved as compound_wallet_features.csv")


100%|████████████████████████████████████████████████████████████████████████████████| 103/103 [06:54<00:00,  4.03s/it]

Feature file saved as compound_wallet_features.csv


In [11]:
# Select features to normalize
feature_cols = ['num_tx', 'total_eth_sent', 'total_eth_received', 
                'compound_tx_count', 'avg_gas_fee', 'tx_duration_days']

scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df[feature_cols]), columns=feature_cols)

# Final score (you can tune weights based on importance)
df['score'] = (
    df_scaled['num_tx'] * 0.2 +
    df_scaled['total_eth_sent'] * 0.2 +
    df_scaled['total_eth_received'] * 0.2 +
    df_scaled['compound_tx_count'] * 0.2 +
    df_scaled['tx_duration_days'] * 0.1 +
    df_scaled['avg_gas_fee'] * 0.1
) * 1000

df['score'] = df['score'].round(0).astype(int)


In [13]:
final_scores = df[['wallet_id', 'score']]
final_scores.to_csv("compound_wallet_scores.csv", index=False)
print("Saved final scores to compound_wallet_scores.csv")


Saved final scores to compound_wallet_scores.csv
